In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import *
import seaborn as sns
import matplotlib.pyplot as plt

### Load data...

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-feb-2021/test.csv")
sample_submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

### Look at data...

In [ ]:
np.sum(train.isnull())

In [ ]:
train.shape

In [ ]:
test.tail()

In [ ]:
train.info()

In [ ]:
sns.pairplot(train.sample(1000))

In [ ]:
sample_submission['target'].value_counts()

In [ ]:
train.corr().style.background_gradient(cmap='orange')

# Do your stuff...

### Deleverables
- EDA
    - What's going on?
    - Show me the data...
    - Bonus: Other / Etc...
- Model
    - [RandomForestRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)
    - Bonus: [KNeighborsRegressor()](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
    - Bonus: Other / Etc...
- Evaluation
    - mean_squared_error()
    - Bonus: Other / Etc...
- RAPIDS Bonus
    - Apply RAPIDS ([Starter Notebook](https://www.kaggle.com/tunguz/tps-feb-2021-rapids-starter))
    - Replace pandas with cuDF & sklearn with cuML

In [ ]:
train.describe

In [ ]:
train.describe(exclude=[np.number])

# ****EDA ::

In [ ]:
%%capture
!pip install category-encoders
import category_encoders as ce
encoder = ce.OrdinalEncoder()
train_encode = encoder.fit_transform(train)

In [ ]:
encoder = ce.OrdinalEncoder()
test_encode = encoder.fit_transform(test)

In [ ]:
train_encode.head()

In [ ]:
test_encode.head()

In [ ]:
sample_submission.tail()

# **Model ::**

In [ ]:
cat_col = [col for col in train.columns if train[col].dtype=='O']
cat_col

In [ ]:
from sklearn.model_selection import train_test_split

X = train_encode.drop('target', axis=1)
y = train_encode.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
cat_col = [col for col in train.columns if train[col].dtype=='O']
cat_col

In [ ]:
categorical = train[cat_col]

categorical_dum = pd.get_dummies(categorical)
categorical_dum

# ****Random Forest Regressor ::

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)

In [ ]:
preds = rfr.predict(X_test)
preds[-4: ]

In [ ]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, preds)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, preds)

# ****K-Neighbors Regressor ::

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=4)
knn.fit(X_train, y_train)

In [ ]:
acc_knn = r2_score(y_test, pred_knn)
acc_knn

In [ ]:
pred_knn = knn.predict(X_test)
pred_knn

# ****Evalation ::

In [ ]:
test_preds = rfr.predict(test_encode)
test_preds[-4: ]

In [ ]:
sample_submission['target'] = test_preds
sample_submission.to_csv('submission.csv', index=False)